huggingface decision transformer:https://huggingface.co/docs/transformers/en/model_doc/decision_transformer

training huggingface decision transformer: https://github.com/huggingface/blog/blob/main/notebooks/101_train-decision-transformers.ipynb


custom gym environments:https://www.gymlibrary.dev/content/environment_creation/

gymnasium documentation: https://gymnasium.farama.org/

decision transformer github: https://github.com/kzl/decision-transformer/tree/master

very simple DT: https://github.com/nikhilbarhate99/min-decision-transformer





In [1]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments

from TensorRTS import TensorRTS
# import tensorRTS

In [5]:
# import random
# import abc
# from typing import Dict, List, Mapping, Tuple, Set
# from entity_gym.env import Observation

# from entity_gym.runner import CliRunner
# from entity_gym.env import *

# class TensorRTS(Environment):
#     """
# LinearRTS, the first epoch of TensorRTS, is intended to be the simplest RTS game.
#     """

#     def __init__(
#         self,
#         mapsize: int = 32,
#         nclusters: int = 6,
#         ntensors: int = 2,
#         maxdots: int = 9,
#         enable_prinouts : bool = True
#     ):
#         self.enable_printouts = enable_prinouts
        
#         if self.enable_printouts:
#             print(f"LinearRTS -- Mapsize: {mapsize}")

#         self.mapsize = mapsize
#         self.maxdots = maxdots
#         self.nclusters = nclusters
#         self.clusters: List[List[int]] = []  # The inner list has a size of 2 (position, number of dots).
#         self.tensors: List[List[int ]] = [] # The inner list has a size of 4 (position, dimension, x, y).

#     def obs_space(cls) -> ObsSpace:
#         return ObsSpace(
#             entities={
#                 "Cluster": Entity(features=["position", "dot"]),
#                 "Tensor": Entity(features=["position", "dimension", "x", "y"]),
#             }
#         )

#     def action_space(cls) -> Dict[ActionName, ActionSpace]:
#         return {
#             "Move": GlobalCategoricalActionSpace(
#                 ["advance", "retreat", "rush", "boom"],
#             ),
#         }

#     def reset(self) -> Observation:

#         #I spent an out baning my head because turns out the reset doesnt delete the clusters from the last game
#         self.clusters: List[List[int]] = []  # The inner list has a size of 2 (position, number of dots).
#         self.tensors: List[List[int ]] = [] # The inner list has a size of 4 (position, dimension, x, y).
#         positions = set()
#         while len(positions) < self.nclusters // 2:
#             position, b = random.choice(
#                 [[position, b] for position in range(self.mapsize // 2) for b in range(1, self.maxdots)]
#             )
#             if position not in positions:
#                 positions.add(position)
#                 self.clusters.append([position, b])
#                 self.clusters.append([self.mapsize - position - 1, b])
#         self.clusters.sort()
 
#         position = random.randint(0, self.mapsize // 2)
#         self.tensors = [[position, 1, 2, 0], [self.mapsize - position - 1, 1, 2, 0]]

#         if self.enable_printouts:
#             self.print_universe()

#         return self.observe()
    
#     def tensor_power(self, tensor_index) -> float :
#         f = self.tensors[tensor_index][3] * self.tensors[tensor_index][3] +  self.tensors[tensor_index][2]
#         if self.enable_printouts:
#             print(f"TP({tensor_index})=TP({self.tensors[tensor_index]})={f}")
#         return f

#     def observe(self) -> Observation:
#         done = self.tensors[0][0] >= self.tensors[1][0]
#         if done:
#             reward = 10 if self.tensor_power(0) > self.tensor_power(1) else 0 if self.tensor_power(0) == self.tensor_power(1) else -10
#         else:
#             reward = 1.0 if self.tensors[0][1] > self.tensors[1][1] else 0.0
#         return Observation(
#             entities={
#                 "Cluster": (
#                     self.clusters,
#                     [("Cluster", i) for i in range(len(self.clusters))],
#                 ),
#                 "Tensor": (
#                     self.tensors,
#                     [("Tensor", i) for i in range(len(self.tensors))],
#                 ),
#             },
#             actions={
#                 "Move": GlobalCategoricalActionMask(),
#             },
#             done=done,
#             reward=reward,
#         )

#     def act(self, actions: Mapping[ActionName, Action], trigger_default_opponent_action : bool = True, is_player_two : bool = False) -> Observation:
#         action = actions["Move"]

#         player_tensor = self.tensors[0]
#         if is_player_two:
#             player_tensor = self.tensors[1]

#         assert isinstance(action, GlobalCategoricalAction)
#         if action.label == "advance" and self.tensors[0][0] < self.mapsize:
#             player_tensor[0] += 1
#             player_tensor[2] += self.collect_dots(player_tensor[0])
#         elif action.label == "retreat" and player_tensor[0] > 0:
#             player_tensor[0] -= 1
#             player_tensor[2] += self.collect_dots(player_tensor[0])
#         elif action.label == "boom":
#             player_tensor[2] += 1
#         elif action.label == "rush":
#             if player_tensor[2] >= 1:
#                 player_tensor[1] = 2 # the number of dimensions is now 2
#                 player_tensor[2] -= 1
#                 player_tensor[3] += 1

#         if trigger_default_opponent_action:
#             self.opponent_act()
        
#         if self.enable_printouts:
#             self.print_universe()

#         return self.observe()

#     def opponent_act(self):         # This is the rush AI.
#         if self.tensors[1][2]>0 :   # Rush if possile
#             self.tensors[1][2] -= 1
#             self.tensors[1][3] += 1
#             self.tensors[1][1] = 2      # the number of dimensions is now 2
#         else:                       # Otherwise Advance.
#             self.tensors[1][0] -= 1
#             self.tensors[1][2] += self.collect_dots(self.tensors[1][0])

#         return self.observe()

#     def collect_dots(self, position):
#         low, high = 0, len(self.clusters) - 1

#         while low <= high:
#             mid = (low + high) // 2
#             current_value = self.clusters[mid][0]

#             if current_value == position:
#                 dots = self.clusters[mid][1]
#                 self.clusters[mid][1] = 0
#                 return dots
#             elif current_value < position:
#                 low = mid + 1
#             else:
#                 high = mid - 1

#         return 0        

#     def print_universe(self):
#         #    print(self.clusters)
#         #    print(self.tensors)
#         for j in range(self.mapsize):
#             print(f" {j%10}", end="")
#         print(" #")
#         position_init = 0
#         for i in range(len(self.clusters)):
#             for j in range(position_init, self.clusters[i][0]):
#                 print("  ", end="")
#             print(f" {self.clusters[i][1]}", end="")
#             position_init = self.clusters[i][0]+1
#         for j in range(position_init, self.mapsize):
#             print("  ", end="")
#         print(" ##")

#         position_init = 0
#         for i in range(len(self.tensors)):
#             for j in range(position_init, self.tensors[i][0]):
#                 print("  ", end="")
#             print(f"{self.tensors[i][2]}", end="")
#             if self.tensors[i][3]>=0:
#                 print(f"-{self.tensors[i][3]}", end="")
#             position_init = self.tensors[i][0]+1
#         for j in range(position_init, self.mapsize):
#             print("  ", end="")
#         print(" ##")

# class Interactive_TensorRTS(TensorRTS): 
#     def __init__(self,
#         mapsize: int = 32,
#         nclusters: int = 6,
#         ntensors: int = 2,
#         maxdots: int = 9, 
#         enable_printouts : bool = True): 
#         self.is_game_over = False

#         super().__init__(mapsize, nclusters, ntensors, maxdots, enable_prinouts=enable_printouts)

#     def act(self, actions: Mapping[ActionName, Action],  trigger_default_opponent_action : bool = True, is_player_two : bool = False, print_universe : bool = False) -> Observation:
#         obs_result = super().act(actions, False, is_player_two)

#         if (obs_result.done == True):
#             self.is_game_over = True

#         return obs_result

# class Agent(metaclass=abc.ABCMeta):
#     def __init__(self, initial_observation : Observation, action_space : Dict[ActionName, ActionSpace]):
#         self.previous_game_state = initial_observation
#         self.action_space = action_space

#     @abc.abstractmethod
#     def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]: 
#         """Pure virtual function in which an agent should return the move that they will make on this turn.

#         Returns:
#             str: name of the action that will be taken
#         """
#         pass

#     @abc.abstractmethod
#     def on_game_start(self) -> None: 
#         """Function which is called for the agent before the game begins.
#         """
#         pass

#     @abc.abstractmethod
#     def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
#         """Function which is called for the agent once the game is over.

#         Args:
#             did_i_win (bool): set to True if this agent won the game.
#         """
#         pass

# class Random_Agent(Agent):
#     def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
#         super().__init__(init_observation, action_space)

#     def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
#         mapping = {}

#         action_choice = random.randrange(0, 2)
#         if (action_choice == 1): 
#             mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[0])
#         else: 
#             mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[1])
        
#         return mapping
    
#     def on_game_start(self) -> None:
#         return super().on_game_start()
    
#     def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
#         return super().on_game_over(did_i_win, did_i_tie)

# class GameResult():
#     def __init__(self, player_one_win : bool = False, player_two_win : bool = False, tie : bool = False):
#         self.player_one_win = player_one_win
#         self.player_two_win = player_two_win
#         self.tie = tie

# class GameRunner(): 
#     def __init__(self, environment = None, enable_printouts : bool = False):
#         self.game = Interactive_TensorRTS(enable_printouts=enable_printouts)
#         self.game.reset()

#         self.player_one = None
#         self.player_two = None
#         self.results : GameResult = None
    
#     def assign_players(self, first_agent : Agent, second_agent : Agent = None):
#         self.player_one = first_agent

#         if second_agent is not None:
#             self.player_two = second_agent

#     def run(self): 
#         assert(self.player_one is not None)

#         game_state = self.game.observe()
#         self.player_one.on_game_start()
#         if self.player_two is not None: 
#             self.player_two.on_game_start()

#         while(self.game.is_game_over is False):
#             #take moves and pass updated environments to agents
#             game_state = self.game.act(self.player_one.take_turn(game_state))
            
#             if (self.game.is_game_over is False):
#                 if self.player_two is None: 
#                     game_state = self.game.opponent_act()
#                 else:
#                     #future player_two code
#                     game_state = self.game.act(self.player_two.take_turn(game_state), False, True)

#         # who won? 
#         tie = False
#         win_p_one = False
#         win_p_two = False

#         p_one = self.game.tensor_power(0)
#         p_two = self.game.tensor_power(1)

#         if p_one > p_two: 
#             win_p_one = True
#         elif p_two > p_one:
#             win_p_two = True
#         else:
#             tie = True

#         self.results = GameResult(win_p_one, win_p_two, tie)
#         self.player_one.on_game_over(win_p_one, tie)
#         if self.player_two is not None:
#             self.player_two.on_game_over(win_p_two, tie)

# # if __name__ == "__main__":  # This is to run wth agents
# #     runner = GameRunner()
# #     init_observation = runner.set_new_game()
# #     random_agent = Random_Agent(init_observation, runner.game.action_space())

# #     runner.assign_players(random_agent)
# #     runner.run()
    
# # if __name__ == "__main__":  #this is to run cli
# #     env = TensorRTS()
# #     # The `CliRunner` can run any environment with a command line interface.
# #     CliRunner(env).run()    

In [61]:
# import gymnasium as gym
# import numpy as np
# from gymnasium import spaces


# class CustomEnv(gym.Env):
#     """Custom Environment that follows gym interface."""

#     metadata = {"render_modes": ["human"], "render_fps": 30}

#     def __init__(self):
#         super().__init__()


#         self.tensor = TensorRTS();
#         # Define action and observation space
#         # They must be gym.spaces objects
#         # Example when using discrete actions:
#         self.action_space = spaces.utils.flatten_space(spaces.Discrete(4))
#         # Example for using image as input (channel-first; channel-last also works):
#         # self.observation_space = spaces.Box(low=0, high=255,
#         #                                     shape=(N_CHANNELS, HEIGHT, WIDTH), dtype=np.uint8)
    
        

#         self.observation_space_unflatten = spaces.Dict({
#              "Cluster": spaces.Box(low=0, high=self.tensor.maxdots, shape=(self.tensor.nclusters, 2),dtype=int),
#              "Tensor": spaces.Box(low=0, high=100, shape=(2, 4), dtype=int)
#             }
#             )
#         self.observation_space = spaces.utils.flatten_space(self.observation_space_unflatten)

#     def step(self, action):
#         ...
#         return observation, reward, terminated, truncated, info

#     def reset(self, seed=None, options=None):
#         super().reset()
#         self.tensor.reset()
#         return self._get_obs()

#     def render(self):
#         ...

#     def close(self):
#         ...


#     def _get_obs(self):
#             return {"Cluster": self.tensor.clusters, "Tensor": self.tensor.tensors}

In [5]:
#thank you ai
import functools
def flatten_dict_of_arrays(data, prefix=""):
  flattened_list = []
  def flatten_inner(value, key):
    if isinstance(value, list):
      for i, v in enumerate(value):
        flatten_inner(v, f"{prefix}{key}[{i}]")
    else:
      flattened_list.append((f"{prefix}{key}", value))

  for key, value in data.items():
    flatten_inner(value, key)

  return [value for _, value in flattened_list]


def value_to_discrete_4(input):
    output = [0,0,0,0]
    output[input] = 1
    return output

In [64]:
#This generates game state data using a random agent vs rush agent
from entity_gym.env import * #for GlobalCategoricalAction

possible_moves = {
    0 : "advance",
    1 : "retreat",
    2 : "rush",
    3 : "boom"
}
rdmEnv = TensorRTS(enable_prinouts = False)
rdmEnv.reset()

rndDataset = {
    'observations': [],
    'actions': [], 
    'rewards': [], 
    'dones': [],
    
}

action = {}
for _ in range(100):
    rdmEnv.reset()

    o = []
    a = []
    r = []
    d = []
    
    for x in range(100):
        o.append(flatten_dict_of_arrays(rdmEnv.observe().features))
        move = random.randrange(0, 4)
        action["Move"] = GlobalCategoricalAction(0, possible_moves[move])
        a.append(value_to_discrete_4(move))

        model_out = rdmEnv.act(action)

        r.append(model_out.reward)

        d.append(model_out.done)
        if model_out.done == True: 
            break 

    rndDataset['observations'].append(o)
    rndDataset['actions'].append(a)
    rndDataset['rewards'].append(r)
    rndDataset['dones'].append(d)
    # rndList.append(rndDataset)

from datasets import Dataset
ds = Dataset.from_dict(rndDataset)


In [77]:
len(ds[1]["observations"][1])

20

In [6]:
# class DecisionTransformerGymDataCollator:
#     return_tensors: str = "pt"
#     max_len: int = 20 #subsets of the episode we use for training
#     state_dim: int = 20  # size of state space
#     act_dim: int = 4  # size of action space
#     max_ep_len: int = 1000 # max episode length in the dataset
#     scale: float = 1000.0  # normalization of rewards/returns
#     state_mean: np.array = None  # to store state means
#     state_std: np.array = None  # to store state stds
#     p_sample: np.array = None  # a distribution to take account trajectory lengths
#     n_traj: int = 0 # to store the number of trajectories in the dataset

#     def __init__(self, dataset) -> None:
#         self.act_dim = len(dataset[0]["actions"][0])
#         self.state_dim = len(dataset[0]["observations"][0])
#         self.dataset = dataset
#         # calculate dataset stats for normalization of states
#         states = []
#         traj_lens = []
#         for obs in dataset["observations"]:
#             states.extend(obs)
#             traj_lens.append(len(obs))
#         self.n_traj = len(traj_lens)
#         states = np.vstack(states)
#         self.state_mean, self.state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        
#         traj_lens = np.array(traj_lens)
#         self.p_sample = traj_lens / sum(traj_lens)

#     def _discount_cumsum(self, x, gamma):
#         discount_cumsum = np.zeros_like(x)
#         discount_cumsum[-1] = x[-1]
#         for t in reversed(range(x.shape[0] - 1)):
#             discount_cumsum[t] = x[t] + gamma * discount_cumsum[t + 1]
#         return discount_cumsum

#     def __call__(self, features):
#         batch_size = len(features)
#         # this is a bit of a hack to be able to sample of a non-uniform distribution
#         batch_inds = np.random.choice(
#             np.arange(self.n_traj),
#             size=batch_size,
#             replace=True,
#             p=self.p_sample,  # reweights so we sample according to timesteps
#         )
#         # a batch of dataset features
#         s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
        
#         for ind in batch_inds:
#             # for feature in features:
#             feature = self.dataset[int(ind)]
#             si = random.randint(0, len(feature["rewards"]) - 1)

#             # get sequences from dataset
#             s.append(np.array(feature["observations"][si : si + self.max_len]).reshape(1, -1, self.state_dim))
#             a.append(np.array(feature["actions"][si : si + self.max_len]).reshape(1, -1, self.act_dim))
#             r.append(np.array(feature["rewards"][si : si + self.max_len]).reshape(1, -1, 1))

#             d.append(np.array(feature["dones"][si : si + self.max_len]).reshape(1, -1))
#             timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
#             timesteps[-1][timesteps[-1] >= self.max_ep_len] = self.max_ep_len - 1  # padding cutoff
#             rtg.append(
#                 self._discount_cumsum(np.array(feature["rewards"][si:]), gamma=1.0)[
#                     : s[-1].shape[1]   # TODO check the +1 removed here
#                 ].reshape(1, -1, 1)
#             )
#             if rtg[-1].shape[1] < s[-1].shape[1]:
#                 print("if true")
#                 rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

#             # padding and state + reward normalization
#             tlen = s[-1].shape[1]
#             s[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, self.state_dim)), s[-1]], axis=1)
#             s[-1] = (s[-1] - self.state_mean) / self.state_std
#             a[-1] = np.concatenate(
#                 [np.ones((1, self.max_len - tlen, self.act_dim)) * -10.0, a[-1]],
#                 axis=1,
#             )
#             r[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), r[-1]], axis=1)
#             d[-1] = np.concatenate([np.ones((1, self.max_len - tlen)) * 2, d[-1]], axis=1)
#             rtg[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), rtg[-1]], axis=1) / self.scale
#             timesteps[-1] = np.concatenate([np.zeros((1, self.max_len - tlen)), timesteps[-1]], axis=1)
#             mask.append(np.concatenate([np.zeros((1, self.max_len - tlen)), np.ones((1, tlen))], axis=1))

#         s = torch.from_numpy(np.concatenate(s, axis=0)).float()
#         a = torch.from_numpy(np.concatenate(a, axis=0)).float()
#         r = torch.from_numpy(np.concatenate(r, axis=0)).float()
#         d = torch.from_numpy(np.concatenate(d, axis=0))
#         rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
#         timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()
#         mask = torch.from_numpy(np.concatenate(mask, axis=0)).float()

#         return {
#             "states": s,
#             "actions": a,
#             "rewards": r,
#             "returns_to_go": rtg,
#             "timesteps": timesteps,
#             "attention_mask": mask,
#         }

In [10]:
# class TrainableDT(DecisionTransformerModel):
#     def __init__(self, config):
#         super().__init__(config)

#     def forward(self, **kwargs):
#         output = super().forward(**kwargs)
#         # add the DT loss
#         action_preds = output[1]
#         action_targets = kwargs["actions"]
#         attention_mask = kwargs["attention_mask"]
#         act_dim = action_preds.shape[2]
#         action_preds = action_preds.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
#         action_targets = action_targets.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        
#         loss = torch.mean((action_preds - action_targets) ** 2)

#         return {"loss": loss}

#     def original_forward(self, **kwargs):
#         return super().forward(**kwargs)

In [65]:
import DTscripts

collator = DTscripts.DecisionTransformerGymDataCollator(ds)
config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
model = DTscripts.TrainableDT(config)

In [69]:

model = DTscripts.TrainableDT.from_pretrained("./output/models4")
    # output_dir="output/",
    # remove_unused_columns=False,
    # num_train_epochs=120,
    # per_device_train_batch_size=64,
    # learning_rate=1e-4,
    # weight_decay=1e-4,
    # warmup_ratio=0.1,
    # optim="adamw_torch",
    # max_grad_norm=0.25,
import wandb
# wandb.init(project="VectorRTS-transformer")

os.environ["WANDB_DISABLED"] = "false" 
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=20,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
    use_mps_device=True,
    # evaluation_strategy="steps",
    # eval_steps = 10

)
training_args = training_args.set_save(strategy="steps", steps=10)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    data_collator=collator,
)

trainer.train()
trainer.save_model("./output/models4")

  0%|          | 0/40 [00:00<?, ?it/s]

{'train_runtime': 5.6067, 'train_samples_per_second': 356.717, 'train_steps_per_second': 7.134, 'train_loss': 0.18081194162368774, 'epoch': 20.0}


In [70]:
trainer.model

TrainableDT(
  (encoder): DecisionTransformerGPT2Model(
    (wte): Embedding(1, 128)
    (wpe): Embedding(1024, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-2): 3 x DecisionTransformerGPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): DecisionTransformerGPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): DecisionTransformerGPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (embed_timestep): Embedding(4096, 128)
  (embed_return): Linear(in_features=1, out_features=128, bias=True)
  (embed_state): Linear(

In [12]:

# model.eval()

In [ ]:
#stolen
# def get_action(model, states, actions, rewards, returns_to_go, timesteps):
#     # This implementation does not condition on past rewards

#     states = states.reshape(1, -1, model.config.state_dim)
#     actions = actions.reshape(1, -1, model.config.act_dim)
#     returns_to_go = returns_to_go.reshape(1, -1, 1)
#     timesteps = timesteps.reshape(1, -1)

#     states = states[:, -model.config.max_length :]
#     actions = actions[:, -model.config.max_length :]
#     returns_to_go = returns_to_go[:, -model.config.max_length :]
#     timesteps = timesteps[:, -model.config.max_length :]
#     padding = model.config.max_length - states.shape[1]
#     # pad all tokens to sequence length
#     attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])])
#     attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
#     states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float()
#     actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float()
#     returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float()
#     timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1)

#     state_preds, action_preds, return_preds = model.original_forward(
#         states=states,
#         actions=actions,
#         rewards=rewards,
#         returns_to_go=returns_to_go,
#         timesteps=timesteps,
#         attention_mask=attention_mask,
#         return_dict=False,
#     )

#     return action_preds[0, -1]

In [3]:


def get_move_no_lookback(state):
    state_dim = 20
    act_dim = 4
    states = torch.from_numpy(state).reshape(1, 1, state_dim).to(device=device, dtype=torch.float32)
    actions = torch.zeros((1, 1, act_dim), device=device, dtype=torch.float32)
    rewards = torch.zeros(1, 1, device=device, dtype=torch.float32)
    target_return = torch.tensor(10, dtype=torch.float32).reshape(1, 1)
    timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
    attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)


    with torch.no_grad():
        state_preds, action_preds, return_preds = model(
            states=states,
            actions=actions,
            rewards=rewards,
            returns_to_go=target_return,
            timesteps=timesteps,
            attention_mask=attention_mask,
            return_dict=False,
            # use_mps_device=True

        )
    return state_preds, action_preds, return_preds


In [7]:
from entity_gym.env import Observation

from entity_gym.env import *
def evaluate_model(model,num_runs):
    testEnv = TensorRTS(enable_prinouts = False)
    possible_moves = {
        0 : "advance",
        1 : "retreat",
        2 : "rush",
        3 : "boom"
    }
    wins = 0
    losses = 0
    ties = 0
    for _ in range(num_runs):

        testEnv.reset()
        for _ in range(1000):
            #make prediction
            model_out = get_move_no_lookback(np.array(flatten_dict_of_arrays(testEnv.observe().features)))
            action['Move'] = GlobalCategoricalAction(0, possible_moves[torch.argmax(model_out[1]).item()])
            pred_reward = model_out[2]
            result = testEnv.act(action)
            # print(model_out[1])

            if result.done == True: 
                break 
            

        # print(result.reward)
        if (result.reward == 10): 
            wins+=1
            print(wins)
        if (result.reward == 0): ties +=1
        if (result.reward == -10): losses+=1


    return ((wins+ties)/num_runs)
model = DecisionTransformerModel.from_pretrained("./output/models4")

evaluate_model(model,100)

0.23

In [2]:
model = DecisionTransformerModel.from_pretrained("./output/models3")
# evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
possible_moves = {
    0 : "advance",
    1 : "retreat",
    2 : "rush",
    3 : "boom"
}
action = {}


testEnv = TensorRTS(enable_prinouts = False)

total_reward = 0

for _ in range(10):

    testEnv.reset()
    for _ in range(100):
        #make prediction
        model_out = get_move_no_lookback(np.array(flatten_dict_of_arrays(testEnv.observe().features)))
        action['Move'] = GlobalCategoricalAction(0, possible_moves[torch.argmax(model_out[1]).item()])
        pred_reward = model_out[2]
        result = testEnv.act(action)
        # print(model_out[1])

        if result.done == True: 
            break 

    total_reward += result.reward

    



total_reward

NameError: name 'get_move_no_lookback' is not defined